In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [47]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [81]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Sex_binario,Embarked_S,Embarked_C,Cabine_nula,Nome_contem_Miss,Nome_contem_Mrs,Nome_contem_Master,Nome_contem_Col,Nome_contem_Major,Nome_contem_Mr
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,0,1,0,1,0,0,0,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,1,0,1,0,0,1,0,0,0,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,1,1,0,1,1,0,0,0,0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,1,1,0,0,0,1,0,0,0,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,0,1,0,1,0,0,0,0,0,1


In [57]:
def transforma_sex(valor):
    if valor == 'female':
        return 1
    else:
        return 0
#Male = 0
#Female = 1

train['Sex_binario'] = train['Sex'].map(transforma_sex)
test['Sex_binario'] = test['Sex'].map(transforma_sex)

In [58]:
train['Embarked_S'] = (train['Embarked'] == 'S').astype(int)
train['Embarked_C'] = (train['Embarked'] == 'C').astype(int)
#train['Embarked_Q'] = (train['Embarked'] == 'Q').astype(int)

train['Cabine_nula'] = train['Cabin'].isnull().astype(int)

train['Nome_contem_Miss'] = train['Name'].str.contains("Miss").astype(int)
train['Nome_contem_Mrs'] = train['Name'].str.contains("Mrs").astype(int)

train['Nome_contem_Master'] = train['Name'].str.contains("Master").astype(int)
train['Nome_contem_Col'] = train['Name'].str.contains("Col").astype(int)
train['Nome_contem_Major'] = train['Name'].str.contains("Major").astype(int)
train['Nome_contem_Mr'] = train['Name'].str.contains("Mr").astype(int)

In [59]:
test['Embarked_S'] = (test['Embarked'] == 'S').astype(int)
test['Embarked_C'] = (test['Embarked'] == 'C').astype(int)
#test['Embarked_Q'] = (test['Embarked'] == 'Q').astype(int)

test['Cabine_nula'] = test['Cabin'].isnull().astype(int)

test['Nome_contem_Miss'] = test['Name'].str.contains("Miss").astype(int)
test['Nome_contem_Mrs'] = test['Name'].str.contains("Mrs").astype(int)

test['Nome_contem_Master'] = test['Name'].str.contains("Master").astype(int)
test['Nome_contem_Col'] = test['Name'].str.contains("Col").astype(int)
test['Nome_contem_Major'] = test['Name'].str.contains("Major").astype(int)
test['Nome_contem_Mr'] = test['Name'].str.contains("Mr").astype(int)

In [60]:
variaveis = ['Sex_binario', 'Age', 'Pclass', 'SibSp', 'Parch', 'Fare', 'Embarked_S', 'Embarked_C', 'Cabine_nula',
            'Nome_contem_Miss', 'Nome_contem_Mrs', 'Nome_contem_Master', 'Nome_contem_Col', 'Nome_contem_Major',
            'Nome_contem_Mr']

X = train[variaveis].fillna(-1)
y = train['Survived']

In [61]:
X.shape, y.shape

((891, 15), (891,))

In [62]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.5)

In [63]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((445, 15), (446, 15), (445,), (446,))

In [64]:
from sklearn.linear_model import LogisticRegression

In [65]:
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

/home/leandro/.conda/envs/Data_Science/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [66]:
p = modelo.predict(X_valid)

In [67]:
from sklearn.metrics import classification_report

In [68]:
print(classification_report(y_valid, p))

              precision    recall  f1-score   support

           0       0.81      0.83      0.82       280
           1       0.70      0.68      0.69       166

    accuracy                           0.77       446
   macro avg       0.76      0.75      0.76       446
weighted avg       0.77      0.77      0.77       446



# Kfold

In [75]:
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold

In [87]:
resultados = []
#Duas divisões, repetidas 10 vezes, com sorteios diferentes e sementes diferentes para cada repetição
kf = RepeatedKFold(n_splits=2, n_repeats=10, random_state=10)

for linhas_treino, linhas_valid in kf.split(X):
        print("Treino: ", linhas_treino.shape[0])
        print("Valid: ", linhas_valid.shape[0])

        X_treino, X_valid = X.iloc[linhas_treino], X.iloc[linhas_valid]
        y_treino, y_valid = y.iloc[linhas_treino], y.iloc[linhas_valid]

        modelo = LogisticRegression()
        modelo.fit(X_treino, y_treino)

        p = modelo.predict(X_valid)

        acc = np.mean(y_valid == p)
        resultados.append(acc)
        print("Acc: ", acc)
        print()

Treino:  445
Valid:  446
Acc:  0.8251121076233184

Treino:  446
Valid:  445
Acc:  0.8044943820224719

Treino:  445
Valid:  446
Acc:  0.8139013452914798

Treino:  446
Valid:  445
Acc:  0.802247191011236

Treino:  445
Valid:  446
Acc:  0.8139013452914798

Treino:  446
Valid:  445
Acc:  0.8179775280898877

Treino:  445
Valid:  446
Acc:  0.8161434977578476

Treino:  446
Valid:  445
Acc:  0.8179775280898877

Treino:  445
Valid:  446
Acc:  0.820627802690583

Treino:  446
Valid:  445
Acc:  0.8089887640449438

Treino:  445
Valid:  446
Acc:  0.827354260089686

Treino:  446
Valid:  445
Acc:  0.8179775280898877

Treino:  445
Valid:  446
Acc:  0.8430493273542601

Treino:  446
Valid:  445
Acc:  0.8067415730337079

Treino:  445
Valid:  446
Acc:  0.8094170403587444

Treino:  446
Valid:  445
Acc:  0.8134831460674158

Treino:  445
Valid:  446
Acc:  0.8452914798206278

Treino:  446
Valid:  445
Acc:  0.8179775280898877

Treino:  445
Valid:  446
Acc:  0.8295964125560538

Treino:  446
Valid:  445


/home/leandro/.conda/envs/Data_Science/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/leandro/.conda/envs/Data_Science/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/leandro/.conda/envs/Data_Science/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/leandro/.conda/envs/Data_Science/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/leandro/.conda/envs/Data_Science/lib/python3.7/site-packages/sklearn/linea

Acc:  0.8292134831460675



In [88]:
np.mean(resultados)

0.8190736635259735

## Testando outro modelo

In [89]:
from sklearn.ensemble import ExtraTreesClassifier

In [90]:
resultados2 = []
#Duas divisões, repetidas 10 vezes, com sorteios diferentes e sementes diferentes para cada repetição
kf = RepeatedKFold(n_splits=2, n_repeats=10, random_state=10)

for linhas_treino, linhas_valid in kf.split(X):
        print("Treino: ", linhas_treino.shape[0])
        print("Valid: ", linhas_valid.shape[0])

        X_treino, X_valid = X.iloc[linhas_treino], X.iloc[linhas_valid]
        y_treino, y_valid = y.iloc[linhas_treino], y.iloc[linhas_valid]

        #modelo = LogisticRegression()
        modelo = ExtraTreesClassifier(max_depth=5, n_estimators=150)
        modelo.fit(X_treino, y_treino)

        p = modelo.predict(X_valid)

        acc = np.mean(y_valid == p)
        resultados2.append(acc)
        print("Acc: ", acc)
        print()

Treino:  445
Valid:  446
Acc:  0.8295964125560538

Treino:  446
Valid:  445
Acc:  0.797752808988764

Treino:  445
Valid:  446
Acc:  0.8340807174887892

Treino:  446
Valid:  445
Acc:  0.7932584269662921

Treino:  445
Valid:  446
Acc:  0.8139013452914798

Treino:  446
Valid:  445
Acc:  0.8112359550561797

Treino:  445
Valid:  446
Acc:  0.8318385650224215

Treino:  446
Valid:  445
Acc:  0.8112359550561797

Treino:  445
Valid:  446
Acc:  0.827354260089686

Treino:  446
Valid:  445
Acc:  0.8

Treino:  445
Valid:  446
Acc:  0.8228699551569507

Treino:  446
Valid:  445
Acc:  0.8292134831460675

Treino:  445
Valid:  446
Acc:  0.8430493273542601

Treino:  446
Valid:  445
Acc:  0.7887640449438202

Treino:  445
Valid:  446
Acc:  0.8094170403587444

Treino:  446
Valid:  445
Acc:  0.8112359550561797

Treino:  445
Valid:  446
Acc:  0.852017937219731

Treino:  446
Valid:  445
Acc:  0.8157303370786517

Treino:  445
Valid:  446
Acc:  0.827354260089686

Treino:  446
Valid:  445
Acc:  0.8292134831460675


In [91]:
np.mean(resultados2)

0.8189560135033004

## Submission

In [69]:
#retreino
modelo = LogisticRegression()
modelo.fit(X,y)

/home/leandro/.conda/envs/Data_Science/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [70]:
p = modelo.predict(test[variaveis].fillna(-1))

In [71]:
sub = pd.Series(p, index=test['PassengerId'], name='Survived')
sub.shape

(418,)

In [73]:
sub.to_csv("Teste_LogisticRegression.csv", header=True)

## Submission 2

In [92]:
#retreino
modelo = ExtraTreesClassifier(max_depth=5, n_estimators=150)
modelo.fit(X,y)
p = modelo.predict(test[variaveis].fillna(-1))

In [93]:
sub = pd.Series(p, index=test['PassengerId'], name='Survived')
sub.shape

(418,)

In [94]:
sub.to_csv("Teste_ExtraTreeClassifier.csv", header=True)